In [3]:
import numpy
import pandas as pd
import numpy as np

data_dir = "../Data/"
data = pd.read_csv(data_dir + "synop.csv")

Quelles sont les stations les plus humides ?
Y a t il une corrélation entre l'humidité et la température ?

In [12]:
humidity=data.groupby('numer_sta')['u'].agg('mean').sort_values(ascending=False)
print(humidity)
correlation=data['u'].corr(data['t'])
print(correlation)

numer_sta
81401    85.643139
7110     84.984380
81405    84.701715
71805    84.369475
61997    83.767886
           ...    
7690     65.687083
7591     62.686538
7747     61.880241
89642    60.163052
7661           NaN
Name: u, Length: 62, dtype: float64
-0.277400971501697


Quels sont les stations qui ont eu le plus de précipitations durant la saison d’hiver et d’automne sur 2023 ? 
Quelle est leur niveau d’exposition au soleil les plus exposés au soleil ?

In [32]:
#Retourne vrai si la date est en hiver ou en automne: pas entre le 19 mars et le 24 septembre
def enAutomneHiver(date):
    return not (319 > int(str(date)[5:8]) > 824)

stations_automne_hiver=pd.DataFrame([data.loc[i, ['numer_sta', 'rr3']] for i in range(len(data)) if enAutomneHiver(data['date'][i]) and data['numer_sta'][i] < 10000])
stations_automne_hiver

,numer_sta,rr3
0,7005.0,-0.1
1,7015.0,-0.1
2,7020.0,3.6
3,7027.0,0.2
4,7037.0,-0.1
...,...,...
190542,7661.0,NaN
190543,7690.0,0.0
190544,7747.0,0.0
190545,7761.0,0.0


In [26]:
data.loc[10, ['numer_sta', 'rr3']]

numer_sta    7149.0
rr3             0.0
Name: 10, dtype: float64

In [39]:
precipiations=stations_automne_hiver.groupby('numer_sta')['rr3'].agg('mean').sort_values(ascending=False)
precipiations

numer_sta
7110.0    0.456322
7434.0    0.404131
7510.0    0.372687
7607.0    0.357643
7591.0    0.348602
7621.0    0.339294
7005.0    0.318964
7627.0    0.313159
7535.0    0.310022
7222.0    0.290384
7117.0    0.288192
7181.0    0.282428
7139.0    0.279366
7027.0    0.276540
7314.0    0.273724
7037.0    0.266284
7015.0    0.261571
7481.0    0.259320
7335.0    0.255923
7020.0    0.245193
7240.0    0.233805
7280.0    0.233365
7255.0    0.232231
7168.0    0.229061
7130.0    0.220173
7072.0    0.219948
7558.0    0.213158
7149.0    0.209651
7471.0    0.198634
7299.0    0.194891
7207.0    0.194807
7630.0    0.194081
7577.0    0.183190
7761.0    0.182714
7190.0    0.166943
7460.0    0.158599
7690.0    0.134097
7643.0    0.096221
7650.0    0.093141
7790.0    0.087635
7747.0    0.077325
7661.0         NaN
Name: rr3, dtype: float64